In [ ]:
import win32com.client
import time
import numpy as np
import matplotlib.pyplot as plt
from astropy.io import fits

In [ ]:
def camera_init():
    try:
        # Launch ASCOM chooser
        chooser = win32com.client.Dispatch("ASCOM.Utilities.Chooser")
        chooser.DeviceType = "Camera"  # Can be 'Telescope', 'Camera', 'FilterWheel', etc.

        # Pick ZWO ASCOM driver (GUI opens)
        camera_id = chooser.Choose(None)
        print("You selected:", camera_id)
        return camera_id
    except:
        print("No camera detected")

In [ ]:
camera_init()

In [ ]:
def take_ramp_qhy(
    cam_name,
    exp_low,
    exp_high,
    num_exp=10,
    num_frames=1,
    img_shape=(1000,1000),
    save_path=None,
    bit_depth=16,
    offset=None
):
    # for QHY42 camera
    import numpy as np
    import time, os
    from astropy.io import fits
    import win32com.client

    # build exposure list
    exp_times = np.logspace(np.log10(exp_low), np.log10(exp_high), num_exp)

    # dtype
    dtype = np.uint16 if bit_depth > 8 else np.uint8

    # output arrays
    img_array = np.zeros((num_exp, num_frames, *img_shape), dtype=dtype)
    img_times = np.zeros((num_exp, num_frames))

    # connect
    cam = win32com.client.Dispatch(cam_name)
    cam.Connected = True

    # read full sensor size once
    full_w = cam.CameraXSize or 4096
    full_h = cam.CameraYSize or 2048
    print(f"Sensor is {full_w}×{full_h}.")

    # force full‐frame in driver
    cam.BinX = cam.BinY = 1
    cam.NumX = full_w
    cam.NumY = full_h
    cam.StartX = 0
    cam.StartY = 0
    if offset is not None:
        cam.Offset = offset

    # compute Python crop offsets
    ty, tx = img_shape
    sy = (full_h - ty)//2
    sx = (full_w - tx)//2

    cam.StartExposure(exp_times[0], False)
    ready=False
    while not ready:
        try:
            _ = cam.ImageArray
            ready = True
        except:
            time.sleep(.01)

    _ = cam.ImageArray

    for i, t in enumerate(exp_times):
        print(f"Exposure {i+1}/{num_exp}: {t:.6f}s")
        for j in range(num_frames):
            cam.StartExposure(t, False)
            ready=False
            while not ready:
                try:
                    full_img = np.array(cam.ImageArray, dtype=dtype)
                    ready = True
                except:
                    time.sleep(.01)

            full_img = np.array(cam.ImageArray, dtype=dtype)
            # ensure full_img has the usual (height, width)
            h, w = full_img.shape
            # expected driver dims
            full_w = cam.CameraXSize or 4096
            full_h = cam.CameraYSize or 2048
            ty, tx = img_shape

            # always crop inside the *left* half:
            if tx > full_w//2:
                raise ValueError("ROI too wide; exceeds one amp region.")

            # center vertically as before
            sy = (full_h - ty)//2
            # center *within the left half* horizontally:
            sx = (full_w//2 - tx)//2

            crop = full_img[sy:sy+ty, sx:sx+tx]


            img_array[i,j] = crop
            img_times[i,j] = time.time()

        # optional save cube
        if save_path:
            os.makedirs(save_path, exist_ok=True)
            cube = img_array[i]
            hdu  = fits.PrimaryHDU(cube)
            hdu = populate_header(cam, hdu, t)
            hdu.writeto(f"{save_path}/image_{i+1:02d}.fits", overwrite=True)

    cam.Connected = False
    return exp_times, img_array, img_times.reshape(-1)


In [ ]:
def take_ramp(cam_name, exp_low, exp_high, num_exp=10, num_frames=1, img_shape=(1000,1000),
              save_path=None, bit_depth=16, temp_setpoint=None, offset=None):

    if cam_name == "ASCOM.QHYCCD.Camera":
        exp_times, img_array, img_times = take_ramp_qhy(cam_name,
        exp_low,
        exp_high,
        num_exp,
        num_frames,
        img_shape,
        save_path,
        bit_depth,
        offset)
    # for regualr ASCOM cameras
    else:
        exp_times = np.logspace(np.log10(exp_low), np.log10(exp_high), num_exp)
        if bit_depth in [10, 12, 14, 16]:
            img_array = np.zeros((num_exp, num_frames, *img_shape), dtype=np.uint16)
        elif bit_depth == 8:
            img_array = np.zeros((num_exp, num_frames, *img_shape), dtype=np.uint8)
        else:
            raise ValueError("Unsupported bit depth. Choose from [8, 10, 12, 14, 16].")
        img_times = np.zeros((num_exp, num_frames))
        # Connect to camera
        camera = win32com.client.Dispatch(cam_name)
        camera.Connected = True
        cam_width = camera.CameraXSize
        cam_height = camera.CameraYSize

        # Add loop to set camera temperature and wait for it to stabilize
        cam_temp = camera.CCDTemperature
        print(f"Camera connected: {cam_name}, Size: {cam_width}x{cam_height}, Temperature: {cam_temp:.2f}C")
        # Setup camera for exposure
        if offset: camera.Offset = offset
        camera.BinX = 1
        camera.BinY = 1
        camera.NumX = img_shape[1]
        camera.NumY = img_shape[0]
        camera.StartX = max(0, (cam_width  - img_shape[1])//2)
        camera.StartY = max(0, (cam_height - img_shape[0])//2)

        for i, exp_time in enumerate(exp_times):
            print(f"Taking frames at {exp_time:.8f} seconds (exposure time {i+1}/{num_exp})")
            for j in range(num_frames):
                camera.StartExposure(exp_time, False)
                while not camera.ImageReady:
                    time.sleep(0.01)
                img = camera.ImageArray
                img_array[i, j] = np.array(img)
                img_times[i, j] = time.time()

            # saves images to 'save_path'
            if save_path is not None:
                img_array3d = img_array[i, :, :, :]
                hdu = fits.PrimaryHDU(img_array3d)
                hdu = populate_header(camera, hdu, exp_time)
                hdul = fits.HDUList([hdu])
                hdul.writeto(f"{save_path}/image_{i+1}.fits", overwrite=True)
        print(f"exp res: {camera.ExposureResolution:.5f} seconds")

        camera.Connected = False
    return  exp_times, img_array, img_times.reshape(-1)

In [ ]:
def populate_header(camera, hdu, exp_time):
    # for creating uniform headers for all fits files
    hdr = hdu.header
    hdr['DATE'] = time.strftime('%Y-%m-%dT%H:%M:%S', time.gmtime())
    hdr.comments['DATE'] = 'File creation date'
    try: hdr['DATE-OBS'] = camera.LastExposureStartTime
    except:
        hdr['DATE-OBS'] = time.strftime('%Y-%m-%dT%H:%M:%S', time.gmtime())
    hdr.comments['DATE-OBS'] = 'Date of data acquisition'
    try: hdr['EXPTIME'] = camera.LastExposureDuration
    except:
        try: hdr['EXPTIME'] = (exp_time//camera.ExposureResolution)*camera.ExposureResolution
        except:
            try: hdr['EXPTIME'] = exp_time[0]
            except:
                try: hdr['EXPTIME'] = exp_time
                except: pass
    hdr.comments['EXPTIME'] = 'Exposure time in s'
    hdr['TEMP'] = camera.CCDTemperature
    hdr.comments['TEMP'] = 'Sensor temperature (in deg C)'
    hdr['MODE'] = camera.ReadoutModes[camera.ReadoutMode]
    hdr['GAIN'] = camera.Gain
    hdr.comments['GAIN'] = 'Camera gain setting if applicable'
    hdr['SENSNAME'] = camera.Name
    return hdu


In [ ]:
# camera = win32com.client.Dispatch("ASCOM.ASICamera2.Camera")
# camera.Connected = True
# print("Connected:", camera.Connected)
# cam_width = camera.CameraXSize
# cam_height = camera.CameraYSize

# # Setup camera for exposure
# camera.BinX = 1
# camera.BinY = 1
# camera.NumX = 1000
# camera.NumY = 1000
# camera.StartX = cam_width // 2 - 500
# camera.StartY = cam_height // 2 - 500

# # Start an exposure (e.g. 1 second, false = light frame)
# exposure_duration = 0.001
# camera.StartExposure(exposure_duration, False)

# # Wait for exposure to finish
# while not camera.ImageReady:
#     time.sleep(0.1)

# # Get image data
# image_data = np.array(camera.ImageArray)  # This will be a 2D array of pixel values

# # Done
# camera.Connected = False
# print("Image captured and camera disconnected.")

In [ ]:
cam_name = "ASCOM.QHYCCD.Camera"
exp_times, img_array, img_times = take_ramp(cam_name, exp_low=.0001, exp_high=0.03, num_exp=60, num_frames=10, img_shape=(1000, 1000), save_path="C:/Users/Jonah/fits_imgs", bit_depth=16)

In [ ]:
mean_vals = np.mean(img_array, axis=(2, 3))  # Mean across frames and image dimensions
plt.plot(exp_times, mean_vals, marker='o')
plt.show()